#### ReportLab use guide

https://www.reportlab.com/docs/reportlab-userguide.pdf

#### ReportLab source code

https://bitbucket.org/rptlab/reportlab/src/998faa6891c038a2453a7828df2be53f7b68ce1d/src/reportlab/?at=default

#### Notes

https://stackoverflow.com/questions/7276017/producing-a-printable-calendar-with-python

https://pairlist2.pair.net/pipermail/reportlab-users/2005-June/004070.html

* Very useful code snippet: 

https://stackoverflow.com/questions/7276017/producing-a-printable-calendar-with-python

* Reportlab: Mixing Fixed Content and Flowables

https://www.blog.pythonlibrary.org/2012/06/27/reportlab-mixing-fixed-content-and-flowables/



## Approach

Approaches to generating calendar pages:

a) Don't use page templates, use just canvas and page breaks

It's possible to draw table on canvas! That's good.

b) Use templates and implement custom flowables to allow custom drawing in the pages

https://www.blog.pythonlibrary.org/2014/03/10/reportlab-how-to-create-custom-flowables/

c) Not sure is possible: use templates and somehow draw on canvas on each page.

Try to somehow access the canvas while creating the story. Maybe some get_canvas method.

d) Use page templates and use global variable to change what's on each page - 

```
def on_page3(canvas, doc):
    global page_data
    canvas.saveState()
    canvas.line(20,20, -20,20)
    canvas.restoreState()
    canvas.drawString(100, 600, f"String on page template 3:{page_data}")
```

I will probably have to use a combination of frames and canvas, because I need tables and tables are flowables. So maybe put a table tightly into a frame. 

## Notes
#### Grid instead of table:
canas.grid(), page 16 of ReportLab manual 

Advantage: it's not flowable, can place it exactly
Disadvantage: how do I put string in the cells? ... tedious

In [202]:
from reportlab.lib.units import inch, cm, mm
from reportlab.lib import colors
from reportlab.lib.papersizes import letter, B5
from reportlab.platypus import SimpleDocTemplate, BaseDocTemplate, PageTemplate, Frame, Spacer, Table, \
TableStyle, Paragraph, NextPageTemplate
from reportlab.graphics.shapes import Drawing
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.pdfgen import canvas
from reportlab.lib.utils import isSeq, encode_label, decode_label, annotateException, strTypes
from reportlab.platypus import Table as pdfTable

import calendar
import datetime

import os
import logging
import copy
from collections import namedtuple


ModuleNotFoundError: No module named 'reportlab.lib.papersizes'

In [199]:
import monthdelta

In [200]:
import reportlab
import pandas as pd

In [203]:
logger = logging.Logger('root', level=logging.DEBUG)
logging.basicConfig()

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter.datefmt = '%m/%d/%Y %I:%M:%S %p' #03/11/2018 11:54:48 AM

#console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(formatter)

logger.addHandler(console_handler)


In [204]:
class Options:
    def __init__(self, **kwds):
        self.__dict__.update(kwds)
    def __str__(self):
        return "".join([ str(key)+":"+str(value)+"\n" for key, value in self.__dict__.items()])

    

In [205]:
reportlab.lib.pagesizes.B5

(500.3149606299212, 708.6614173228346)

In [305]:
def fill_options():
    
    debug_verbosity = 1
    
    layout=1 
    #format={
    #    'start_date' : datetime.date(2018,10,1),
    #    'end_date' : datetime.date(2019,6,4)
    #}
    daterange={
                'start_date' : datetime.date(2018,5,1),
                #'start_date' : format['start_date'],
                'end_date' : datetime.date(2019,2,10),
                #'end_date' : format['end_date']
        
              }
    monthrange={
                'start' : daterange['start_date'],
                'stop' : daterange['end_date']
                }
    # diary structure as a mapping pages->sections
    # Is generated from diary_structure_template
    # diary_structure_template -> diary_sections

    sections = ['info_sheet'] + ['months_v1_landscape_cutout'] + 4*['notes_v1_landscape_cutout'] + \
               ['weeks_v2_landscape_cutout'] + 4*['notes_v1_landscape_cutout']

    papersize = [t for t in reversed(reportlab.lib.pagesizes.A4)] # landscape, size for printing on paper
    pagesize = (15.7*cm, papersize[1]) # size for drawing and cutting
    margins = tuple(4*[0.5*cm]) #left, right, upper, lower

    return Options(
            debug_verbosity = debug_verbosity,
            layout=layout, 
            format=format, 
            daterange=daterange, 
            monthrange=monthrange, 
            sections=sections,
            pagesize=pagesize,
            papersize=papersize,
            margins=margins)


o = fill_options()

In [207]:
height = o.papersize[1]
width = o.papersize[0]
pheight = o.pagesize[1] # final paper height
pwidth = o.pagesize[0] # final paper width

In [258]:
logger.info(o)

06/08/2018 06:18:43 PM - root - INFO - debug_verbosity:1
layout:1
format:<built-in function format>
daterange:{'start_date': datetime.date(2018, 7, 1), 'end_date': datetime.date(2019, 2, 10)}
monthrange:{'start': datetime.date(2018, 7, 1), 'stop': datetime.date(2019, 2, 10)}
sections:['info_sheet', 'months_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'weeks_v2_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout']
pagesize:(445.03937007874015, 595.275590551181)
papersize:[841.8897637795275, 595.275590551181]
margins:(14.173228346456693, 14.173228346456693, 14.173228346456693, 14.173228346456693)



In [209]:
# global variable VERBOSITY
VERBOSITY = o.debug_verbosity

In [210]:
# fix this 
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
days_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [211]:
def nearest_past_Monday(initial_date):
# find the date of the nearet present or past Monday
    logger.debug(f'Initial date:{initial_date}, {weekdays[initial_date.weekday()]}')
    for i in range(7):
        # iterate 0 to 6
        previous = initial_date - datetime.timedelta(days=i)
        day = weekdays[previous.weekday()]
        logger.debug(f'{previous}, {day}')
        if day == 'Monday':
            return previous
        
    assert(False), "Something wrong. Iterated back 7 days and didn't find a Monday."
            
if 1:
    print(nearest_past_Monday( datetime.date(2018,5,8)))
    print(nearest_past_Monday( datetime.date(2018,5,7)))
    print(nearest_past_Monday( datetime.date(2018,5,6)))
    

06/08/2018 05:35:11 PM - root - DEBUG - Initial date:2018-05-08, Tuesday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-05-08, Tuesday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-05-07, Monday
06/08/2018 05:35:11 PM - root - DEBUG - Initial date:2018-05-07, Monday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-05-07, Monday
06/08/2018 05:35:11 PM - root - DEBUG - Initial date:2018-05-06, Sunday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-05-06, Sunday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-05-05, Saturday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-05-04, Friday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-05-03, Thursday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-05-02, Wednesday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-05-01, Tuesday
06/08/2018 05:35:11 PM - root - DEBUG - 2018-04-30, Monday


2018-05-07
2018-05-07
2018-04-30


In [212]:
def nearest_future_Sunday(initial_date):
# find the date of the nearet present or future Sunday
    logger.debug(f'Initial date:, {initial_date}, {weekdays[initial_date.weekday()]}')
    for i in range(7):
        # iterate 0 to 6
        next = initial_date + datetime.timedelta(days=i)
        day = weekdays[next.weekday()]
        logger.debug(f'{next},{day}')
        if day == 'Sunday':
            return next
        
    assert(False), "Something wrong. Iterated back 7 days and didn't find a Monday."
            
if 1:
    print(nearest_future_Sunday( datetime.date(2018,5,5)))
    print(nearest_future_Sunday( datetime.date(2018,5,6)))
    print(nearest_future_Sunday( datetime.date(2018,5,7)))

06/08/2018 05:35:19 PM - root - DEBUG - Initial date:, 2018-05-05, Saturday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-05,Saturday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-06,Sunday
06/08/2018 05:35:19 PM - root - DEBUG - Initial date:, 2018-05-06, Sunday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-06,Sunday
06/08/2018 05:35:19 PM - root - DEBUG - Initial date:, 2018-05-07, Monday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-07,Monday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-08,Tuesday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-09,Wednesday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-10,Thursday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-11,Friday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-12,Saturday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-05-13,Sunday


2018-05-06
2018-05-06
2018-05-13


In [213]:
def get_calendar_boundaries(start_date, end_date):
    # return the date of the nearest previous Monday and the date of the nearest next Sunday in a tuple

    assert(start_date<=end_date), "Start day must be earlier or the same as end date"

    diary_first_date = nearest_past_Monday(start_date)
    diary_last_date = nearest_future_Sunday(end_date)

    # check it's at least 1 week
    assert( (diary_last_date - diary_first_date).days >= 6 )

    # check it's exactly an integer multiple of 7
    assert( ((diary_last_date - diary_first_date).days+1)//7 - ((diary_last_date - diary_first_date).days+1)/7 == 0 ), "It's not whole weeks"

    # check that the first day is Monday
    assert( diary_first_date.weekday()==0 ), "First date must be Monday"
    # check that the last day is Sunday
    assert( diary_last_date.weekday()==6 ), "Last date must be Sunday"
    
    return (diary_first_date, diary_last_date)

if 1:
    print(o)
    print( get_calendar_boundaries(o.daterange['start_date'], o.daterange['end_date'] ))

06/08/2018 05:35:19 PM - root - DEBUG - Initial date:2018-07-01, Sunday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-07-01, Sunday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-06-30, Saturday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-06-29, Friday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-06-28, Thursday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-06-27, Wednesday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-06-26, Tuesday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-06-25, Monday
06/08/2018 05:35:19 PM - root - DEBUG - Initial date:, 2019-02-10, Sunday
06/08/2018 05:35:19 PM - root - DEBUG - 2019-02-10,Sunday


debug_verbosity:1
layout:1
format:<built-in function format>
daterange:{'start_date': datetime.date(2018, 7, 1), 'end_date': datetime.date(2019, 2, 10)}
monthrange:{'start': datetime.date(2018, 7, 1), 'stop': datetime.date(2019, 2, 10)}
sections:['info_sheet', 'months_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'weeks_v2_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout']
pagesize:(445.03937007874015, 595.275590551181)
papersize:[841.8897637795275, 595.275590551181]
margins:(14.173228346456693, 14.173228346456693, 14.173228346456693, 14.173228346456693)

(datetime.date(2018, 6, 25), datetime.date(2019, 2, 10))


In [214]:
def get_month(year, month):
    
    cal = calendar.Calendar()
    month_data = []
    for d in cal.itermonthdays2(year, month):
        #print(d[0], days_names[d[1]])
        if d[0]!=0:
            cell = str(d[0])+(len(str(d[0]))==1)*" "+" "+days_names[d[1]]
            month_data.append(cell)

    month_data = [ [i, ""] for i in month_data]
    return month_data

if 1:
    print( get_month(2018, 5) )

[['1  Tue', ''], ['2  Wed', ''], ['3  Thu', ''], ['4  Fri', ''], ['5  Sat', ''], ['6  Sun', ''], ['7  Mon', ''], ['8  Tue', ''], ['9  Wed', ''], ['10 Thu', ''], ['11 Fri', ''], ['12 Sat', ''], ['13 Sun', ''], ['14 Mon', ''], ['15 Tue', ''], ['16 Wed', ''], ['17 Thu', ''], ['18 Fri', ''], ['19 Sat', ''], ['20 Sun', ''], ['21 Mon', ''], ['22 Tue', ''], ['23 Wed', ''], ['24 Thu', ''], ['25 Fri', ''], ['26 Sat', ''], ['27 Sun', ''], ['28 Mon', ''], ['29 Tue', ''], ['30 Wed', ''], ['31 Thu', '']]


In [215]:
def get_days(options):
    '''Return a DataFrame with days for the weeks section'''
    
    # iterate through days of the diary
    (first_date, last_date) = \
        get_calendar_boundaries(options.daterange['start_date'], options.daterange['end_date'] )
    print(first_date, last_date)

    df_diary = pd.DataFrame()

    logger.debug("Days of the diary:")
    day = first_date
    week_index = 0
    i = 0
    while day<=last_date:
        df_diary = df_diary.append({'week_index': int(week_index), 'date': day, 'weekday': weekdays[day.weekday()]}, ignore_index=True)

        logger.debug(f'{week_index}, {day}, {weekdays[day.weekday()]}')
        day += datetime.timedelta(days=1)

        i += 1
        if i % 7 ==0:
            week_index += 1

    df_diary['week_index'] = df_diary['week_index'].astype(int)
    df_diary['week_number'] =  df_diary['date'].apply(lambda x: x.isocalendar()[1])
    df_diary['month_number'] =  df_diary['date'].apply(lambda x: x.month)
    df_diary['month_name'] = df_diary['date'].apply(lambda x: x.strftime("%B"))
    
    return df_diary

if 1:
    test_options = copy.deepcopy(o)
    test_options.daterange['start_date']=datetime.date(2018, 12, 24)
    test_options.daterange['end_date']=datetime.date(2019, 6, 10)
    
    df_days = get_days(options=test_options)
    display(df_days)

06/08/2018 05:35:19 PM - root - DEBUG - Initial date:2018-12-24, Monday
06/08/2018 05:35:19 PM - root - DEBUG - 2018-12-24, Monday
06/08/2018 05:35:19 PM - root - DEBUG - Initial date:, 2019-06-10, Monday
06/08/2018 05:35:19 PM - root - DEBUG - 2019-06-10,Monday
06/08/2018 05:35:19 PM - root - DEBUG - 2019-06-11,Tuesday
06/08/2018 05:35:19 PM - root - DEBUG - 2019-06-12,Wednesday
06/08/2018 05:35:19 PM - root - DEBUG - 2019-06-13,Thursday
06/08/2018 05:35:20 PM - root - DEBUG - 2019-06-14,Friday
06/08/2018 05:35:20 PM - root - DEBUG - 2019-06-15,Saturday
06/08/2018 05:35:20 PM - root - DEBUG - 2019-06-16,Sunday
06/08/2018 05:35:20 PM - root - DEBUG - Days of the diary:
06/08/2018 05:35:20 PM - root - DEBUG - 0, 2018-12-24, Monday
06/08/2018 05:35:20 PM - root - DEBUG - 0, 2018-12-25, Tuesday
06/08/2018 05:35:20 PM - root - DEBUG - 0, 2018-12-26, Wednesday
06/08/2018 05:35:20 PM - root - DEBUG - 0, 2018-12-27, Thursday
06/08/2018 05:35:20 PM - root - DEBUG - 0, 2018-12-28, Friday
06/08/

2018-12-24 2019-06-16


06/08/2018 05:35:20 PM - root - DEBUG - 2, 2019-01-13, Sunday
06/08/2018 05:35:20 PM - root - DEBUG - 3, 2019-01-14, Monday
06/08/2018 05:35:20 PM - root - DEBUG - 3, 2019-01-15, Tuesday
06/08/2018 05:35:20 PM - root - DEBUG - 3, 2019-01-16, Wednesday
06/08/2018 05:35:20 PM - root - DEBUG - 3, 2019-01-17, Thursday
06/08/2018 05:35:20 PM - root - DEBUG - 3, 2019-01-18, Friday
06/08/2018 05:35:20 PM - root - DEBUG - 3, 2019-01-19, Saturday
06/08/2018 05:35:20 PM - root - DEBUG - 3, 2019-01-20, Sunday
06/08/2018 05:35:20 PM - root - DEBUG - 4, 2019-01-21, Monday
06/08/2018 05:35:20 PM - root - DEBUG - 4, 2019-01-22, Tuesday
06/08/2018 05:35:20 PM - root - DEBUG - 4, 2019-01-23, Wednesday
06/08/2018 05:35:20 PM - root - DEBUG - 4, 2019-01-24, Thursday
06/08/2018 05:35:20 PM - root - DEBUG - 4, 2019-01-25, Friday
06/08/2018 05:35:20 PM - root - DEBUG - 4, 2019-01-26, Saturday
06/08/2018 05:35:20 PM - root - DEBUG - 4, 2019-01-27, Sunday
06/08/2018 05:35:20 PM - root - DEBUG - 5, 2019-01-28,

06/08/2018 05:35:21 PM - root - DEBUG - 21, 2019-05-22, Wednesday
06/08/2018 05:35:21 PM - root - DEBUG - 21, 2019-05-23, Thursday
06/08/2018 05:35:21 PM - root - DEBUG - 21, 2019-05-24, Friday
06/08/2018 05:35:21 PM - root - DEBUG - 21, 2019-05-25, Saturday
06/08/2018 05:35:21 PM - root - DEBUG - 21, 2019-05-26, Sunday
06/08/2018 05:35:21 PM - root - DEBUG - 22, 2019-05-27, Monday
06/08/2018 05:35:21 PM - root - DEBUG - 22, 2019-05-28, Tuesday
06/08/2018 05:35:21 PM - root - DEBUG - 22, 2019-05-29, Wednesday
06/08/2018 05:35:21 PM - root - DEBUG - 22, 2019-05-30, Thursday
06/08/2018 05:35:21 PM - root - DEBUG - 22, 2019-05-31, Friday
06/08/2018 05:35:21 PM - root - DEBUG - 22, 2019-06-01, Saturday
06/08/2018 05:35:21 PM - root - DEBUG - 22, 2019-06-02, Sunday
06/08/2018 05:35:21 PM - root - DEBUG - 23, 2019-06-03, Monday
06/08/2018 05:35:21 PM - root - DEBUG - 23, 2019-06-04, Tuesday
06/08/2018 05:35:21 PM - root - DEBUG - 23, 2019-06-05, Wednesday
06/08/2018 05:35:21 PM - root - DEBU

,date,week_index,weekday,week_number,month_number,month_name
0,2018-12-24,0,Monday,52,12,December
1,2018-12-25,0,Tuesday,52,12,December
2,2018-12-26,0,Wednesday,52,12,December
3,2018-12-27,0,Thursday,52,12,December
4,2018-12-28,0,Friday,52,12,December
5,2018-12-29,0,Saturday,52,12,December
6,2018-12-30,0,Sunday,52,12,December
7,2018-12-31,1,Monday,1,12,December
8,2019-01-01,1,Tuesday,1,1,January
9,2019-01-02,1,Wednesday,1,1,January


In [216]:
class Counters():
    def __init__(self):
        self.page = 0
        self.week = 0
    def reset(self):
        self.page = 0
        self.week = 0
        


In [217]:
counters = Counters()

In [218]:
def footer(canvas, s):
    canvas.drawString(pwidth/2, 0, s)

In [219]:
def debug_print(x, y, s, canvas):
    if VERBOSITY == 0:
        pass
    elif VERBOSITY == 1:
        canvas.drawString(x, y, s)
    else:
        assert(False), f'Unknown value of VERBOSITY:{VERBOSITY}'
        

In [250]:
def draw_paperframe(mycanvas, options, offset=0):
    # draw a rectangle around wht should be cut
    x0,y0 = offset,0
    mycanvas.setDash([3,3],0)

    mycanvas.rect(x0,y0,x0+pwidth,y0+pheight, stroke=1)

In [266]:
o.papersize[0]


841.8897637795275

In [311]:
def get_offset(page_number, options):
    margin = 20
    if page_number % 2 == 0: # zero od even
        # left placement
        return margin+0
    else:
        # right placement
        #return margin+0
        # for double-sided printers:
        return options.papersize[0]-options.pagesize[0]-margin
    

In [304]:
def gen_months_page_landscape(options, first_month, second_month, mycanvas):
    
    '''
    first_month is tuple (year, month)
    second_month is tuple (year, month)
    '''
    
    global counters
    
    logger.info(f"Generating months {first_month} and {second_month}")
    
    logger.info(f"Page number:{counters.page}")
    
    debug_print(0,height-10,"Months section", mycanvas)
    #mycanvas.drawString(0,height-10,"Months section")
    
    offset = get_offset(counters.page, options)
        
    draw_paperframe(mycanvas, options, offset)

    common_style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        #('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        ('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        ('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]

    # to-do: use a fix width font

    mycanvas.drawString(offset+10,height-20,f"{calendar.month_name[first_month[1]]} {first_month[0]}")

    month_data = get_month(first_month[0],first_month[1])
    
    table = Table(month_data, colWidths=[20*mm,46*mm]) # altogether 66mm

    # indices: column, row, top left is 0,0, bottom right is -1,-1

    # mark weekends
    style = copy.deepcopy( common_style )
    style += [ ('FONT', (0,i),(0,i), 'Helvetica-Bold' ) for i,d in enumerate(month_data) if ("Sat" in d[0]) or ("Sun" in d[0]) ]

    #logger.debug(style)
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    table.drawOn(mycanvas, offset+10, 10)

    # ------- second month on a page
    
    month_data = get_month(second_month[0],second_month[1])

    mycanvas.drawString(offset+250,height-20,f"{calendar.month_name[second_month[1]]} {second_month[0]}")

    table = Table(month_data, colWidths=[50,150])

    # indices: column, row, top left is 0,0, bottom right is -1,-1

    # mark weekends
    style = copy.deepcopy( common_style )
    style += [ ('FONT', (0,i),(0,i), 'Helvetica-Bold' ) for i,d in enumerate(month_data) if ("Sat" in d[0]) or ("Sun" in d[0]) ]

    #logger.debug(style)
    
    table.setStyle(TableStyle(style))
    w, h2 = table.wrapOn(mycanvas, 0, 0)
    table.drawOn(mycanvas, offset+250, 10+h-h2) # use previous height h so that they are aligned at the top 
    
    footer(mycanvas, str(counters.page))
    mycanvas.showPage() 
    counters.page += 1
    
if 0:
    VERBOSITY = 1
    counters.reset()
    c = canvas.Canvas('test.pdf', pagesize = o.papersize)
    gen_months_page_landscape(o, (2018,12), (2019,1), c)
    c.save()
    os.system("start " + 'test.pdf')
    
# test layout across opposite sides of a page
if 1:
    VERBOSITY = 1
    counters.reset()
    c = canvas.Canvas('test.pdf', pagesize = o.papersize)
    gen_months_page_landscape(o, (2018,12), (2019,1), c)
    gen_months_page_landscape(o, (2019,2), (2019,3), c)
    c.save()
    os.system("start " + 'test.pdf')
    
    

06/08/2018 06:48:00 PM - root - INFO - Generating months (2018, 12) and (2019, 1)
06/08/2018 06:48:00 PM - root - INFO - Page number:0
06/08/2018 06:48:00 PM - root - INFO - Generating months (2019, 2) and (2019, 3)
06/08/2018 06:48:00 PM - root - INFO - Page number:1


In [222]:
def gen_months_page_portrait(options, first_month, second_month, mycanvas):
    
    '''
    first_month is tuple (year, month)
    second_month is tuple (year, month)
    '''
    
    global counters
    
    logger.info(f"Generating months {first_month} and {second_month}")
    
    logger.info(f"Page number:{counters.page}")
    
    debug_print(0,height-10,"Months section", mycanvas)
    #mycanvas.drawString(0,height-10,"Months section")

    common_style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        #('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        ('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        ('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]

    # to-do: use a fix width font

    mycanvas.drawString(10,height-20,f"{calendar.month_name[first_month[1]]} {first_month[0]}")

    month_data = get_month(first_month[0],first_month[1])
    
    table = Table(month_data, colWidths=[20*mm,46*mm]) # altogether 66mm

    # indices: column, row, top left is 0,0, bottom right is -1,-1

    # mark weekends
    style = copy.deepcopy( common_style )
    style += [ ('FONT', (0,i),(0,i), 'Helvetica-Bold' ) for i,d in enumerate(month_data) if ("Sat" in d[0]) or ("Sun" in d[0]) ]

    #logger.debug(style)
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    table.drawOn(mycanvas, 10, 50)

    # ------- second month on a page
    
    month_data = get_month(second_month[0],second_month[1])

    mycanvas.drawString(250,height-20,f"{calendar.month_name[second_month[1]]} {second_month[0]}")

    table = Table(month_data, colWidths=[50,150])

    # indices: column, row, top left is 0,0, bottom right is -1,-1

    # mark weekends
    style = copy.deepcopy( common_style )
    style += [ ('FONT', (0,i),(0,i), 'Helvetica-Bold' ) for i,d in enumerate(month_data) if ("Sat" in d[0]) or ("Sun" in d[0]) ]

    #logger.debug(style)
    
    table.setStyle(TableStyle(style))
    w, h2 = table.wrapOn(mycanvas, 0, 0)
    table.drawOn(mycanvas, 250, 50+h-h2) # use previous height h so that they are aligned at the top 
    
    footer(mycanvas, str(counters.page))
    mycanvas.showPage() 
    counters.page += 1
    
if 1:
    VERBOSITY = 0
    counters.reset()
    c = canvas.Canvas('test.pdf', pagesize=o.pagesize)
    gen_months_page_portrait(o, (2018,12), (2019,1), c)
    c.save()
    os.system("start " + 'test.pdf')

06/08/2018 05:35:22 PM - root - INFO - Generating months (2018, 12) and (2019, 1)
06/08/2018 05:35:22 PM - root - INFO - Page number:0


In [223]:
from dateutil.rrule import rrule, MONTHLY

In [224]:
def get_months_list(start_year, start_month, end_year, end_month):
    # return list of months
    # [(12, 2018), (1, 2019), (2, 2019), (3, 2019)]
    start = datetime.datetime(start_year, start_month, 1)
    end = datetime.datetime(end_year, end_month, 1)
    return [(d.year, d.month) for d in rrule(MONTHLY, dtstart=start, until=end)]

In [225]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [284]:
def gen_months_section(options, canvas):
    
    start_month = options.monthrange['start']
    last_month = options.monthrange['stop']
    
    logger.debug(start_month)
    logger.debug(last_month)
    
    number_of_months = diff_month(last_month, start_month) + 1
    
    logger.info(f"Number of months = {number_of_months}")
    
    assert(number_of_months>0), "Zero number of months is not allowed."
    
    if number_of_months % 2 == 0: #even number of months because there must be two on page
        logger.info("Even number of months.")
        pass
    else: # odd number of months, add one month
        logger.debug("Odd number of months, adding one month.")
        old_last_month = last_month
        last_month += monthdelta.monthdelta(1)
        logger.debug(f"{old_last_month} -> {last_month}")
    
    months = get_months_list(start_month.year, start_month.month, last_month.year, last_month.month)
    # e.g., [(12, 2018), (1, 2019), (2, 2019), (3, 2019)]
    
    logger.info(f"Months section generated for the following list of months:{months}")
    
    for i, m in enumerate(months[::2]): # iterate by two steps
        logger.debug(f"{i, i*2, i*2+1}, {m}")
        logger.debug(m) # first month on the page
        logger.debug(months[i*2+1]) # second month on the page
        gen_months_page_landscape(options, m, months[i*2+1], canvas)
    
if 1:
    logger.setLevel(logging.DEBUG)
    
    counters.reset()
    
    c = canvas.Canvas('test.pdf', pagesize = o.papersize)
   
    test_options = copy.deepcopy(o)
    test_options.monthrange['start'] = datetime.date(2018,5,15)
    test_options.monthrange['stop'] = datetime.date(2019,3,1)
    
    print(test_options)
    
    gen_months_section(test_options, c)
    c.save()
    os.system("start " + 'test.pdf')    
    

06/08/2018 06:25:10 PM - root - DEBUG - 2018-05-15
06/08/2018 06:25:10 PM - root - DEBUG - 2019-03-01
06/08/2018 06:25:10 PM - root - INFO - Number of months = 11
06/08/2018 06:25:10 PM - root - DEBUG - Odd number of months, adding one month.
06/08/2018 06:25:10 PM - root - DEBUG - 2019-03-01 -> 2019-04-01
06/08/2018 06:25:10 PM - root - INFO - Months section generated for the following list of months:[(2018, 5), (2018, 6), (2018, 7), (2018, 8), (2018, 9), (2018, 10), (2018, 11), (2018, 12), (2019, 1), (2019, 2), (2019, 3), (2019, 4)]
06/08/2018 06:25:10 PM - root - DEBUG - (0, 0, 1), (2018, 5)
06/08/2018 06:25:10 PM - root - DEBUG - (2018, 5)
06/08/2018 06:25:10 PM - root - DEBUG - (2018, 6)
06/08/2018 06:25:10 PM - root - INFO - Generating months (2018, 5) and (2018, 6)
06/08/2018 06:25:10 PM - root - INFO - Page number:0
06/08/2018 06:25:10 PM - root - DEBUG - (1, 2, 3), (2018, 7)
06/08/2018 06:25:10 PM - root - DEBUG - (2018, 7)
06/08/2018 06:25:10 PM - root - DEBUG - (2018, 8)
06/

debug_verbosity:1
layout:1
format:<built-in function format>
daterange:{'start_date': datetime.date(2018, 7, 1), 'end_date': datetime.date(2019, 2, 10)}
monthrange:{'start': datetime.date(2018, 5, 15), 'stop': datetime.date(2019, 3, 1)}
sections:['info_sheet', 'months_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'weeks_v2_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout']
pagesize:(445.03937007874015, 595.275590551181)
papersize:[841.8897637795275, 595.275590551181]
margins:(14.173228346456693, 14.173228346456693, 14.173228346456693, 14.173228346456693)



In [281]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
    

C:\Users\D\.virtualenvs\Calendar-CphqioUX\Scripts\python.exe
3.6.0 (v3.6.0:41df79263a11, Dec 23 2016, 08:06:12) [MSC v.1900 64 bit (AMD64)]
sys.version_info(major=3, minor=6, micro=0, releaselevel='final', serial=0)


In [293]:
def gen_week_landscape(options, week, mycanvas):
    ''' Generate a single week
    
    Current design is 1 week on 2 pages.
    
    week ... list of days in the week, first is Monday, last is Sunday
    [datetime.date(2018, 4, 30), datetime.date(2018, 5, 1), datetime.date(2018, 5, 2), 
     datetime.date(2018, 5, 3), datetime.date(2018, 5, 4), datetime.date(2018, 5, 5), datetime.date(2018, 5, 6)]
    
    week=
             date       week_index  weekday      week_number month_number month_name
        0  2018-12-24           0     Monday           52            12   December
        1  2018-12-25           0    Tuesday           52            12   December
        2  2018-12-26           0  Wednesday           52            12   December
        3  2018-12-27           0   Thursday           52            12   December
        4  2018-12-28           0     Friday           52            12   December
        5  2018-12-29           0   Saturday           52            12   December
        6  2018-12-30           0     Sunday           52            12   December
    
    '''
    
    global counters
    
    logger.debug(week)
    
    offset = get_offset(counters.page, options)
    
    draw_paperframe(mycanvas, options, offset) 
    
    week_section_options = {
        'top_margin' : 9*mm,
        'bottom_margin' : 9*mm,
        'top_box_height' : 3*cm,
        'weekly_tasks_width' : 5*cm,
        'weekly_tasks_row_height' : 6*mm,
        'weekly_focus_height' : 5*cm,
        'day_box_width' : 7*cm,
        'day_box_nlines' : 4, # number of lines inside each day box 
    }
    
    debug_print(0,height-10,f"Week section, week={week.date.iloc[0]}-{week.date.iloc[-1]}", mycanvas)
    
    mycanvas.drawString(offset+0, height-20, f"Week number:{week.week_number.iloc[0]}")
    
    # boxes for days of the week
    day_box_height =( pheight-week_section_options['top_margin']-week_section_options['bottom_margin'] ) / 7
        
    # spacing of lines inside each day box
    day_box_line_spacing = day_box_height / (week_section_options['day_box_nlines']+1)
    logger.debug(day_box_line_spacing)
        
    for i, (ind, day) in enumerate( week.iterrows() ):
        day_name = day['weekday']
        logger.debug(f"{day.date}:{day_name}")
        mycanvas.grid([offset+week_section_options['weekly_tasks_width'], 
                       offset+pwidth
                      ],
                      [height-week_section_options['top_margin']-i*day_box_height, 
                       height-week_section_options['top_margin']-(i+1)*day_box_height])
        # name of the day in the day box
        mycanvas.drawString(offset+week_section_options['weekly_tasks_width'], 
                            height-week_section_options['top_margin']-i*day_box_height-(1)*day_box_line_spacing+1*mm,
                            f"{day_name[0:2]}:{day.date.day}.{day.date.month}.")

                                                 
        mycanvas.setStrokeColor('pink')
        for j in range( week_section_options['day_box_nlines'] ):
            mycanvas.line(offset+week_section_options['weekly_tasks_width'],
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing, 
                          offset+week_section_options['weekly_tasks_width'] + (pwidth-week_section_options['weekly_tasks_width'])/2-0.5*cm,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing 
                         )
            mycanvas.line(offset+week_section_options['weekly_tasks_width']+(pwidth-week_section_options['weekly_tasks_width'])/2+0.5*cm,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing, 
                          offset+pwidth,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing 
                         )
        mycanvas.setStrokeColor('black')
                      
    
    # top box
    
    table = Table([[""]], colWidths=week_section_options['weekly_tasks_width'], 
                          rowHeights=week_section_options['top_box_height'])
    
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 1, colors.gray, None, (1,4)),
        ('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ]
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table width, height is {w, h}")
    table.drawOn(mycanvas, offset+0, height-week_section_options['top_box_height'])
    
    
    # left column with week tasks
    
    common_style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        #('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        ('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        ('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]

    # to-do: use a fix width font

    
    nr = int( (height-week_section_options['top_box_height']-week_section_options['weekly_focus_height'])//(6*mm) )

    table = Table(data=[ [""] for i in range(int(nr))], 
                  colWidths=[5*cm], rowHeights=nr*[week_section_options['weekly_tasks_row_height']]) 
    
    table.setStyle(TableStyle(common_style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table height is {h}")
    table.drawOn(mycanvas, offset+0, week_section_options['weekly_focus_height'])
    
    
    # weekly focus subsection
    table = Table([[""]], colWidths=week_section_options['weekly_tasks_width'], 
                          rowHeights=week_section_options['weekly_focus_height'])
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0),
          (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 1, colors.gray, None, (1,4)),
        ('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ]
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table width, height is {w, h}")
    table.drawOn(mycanvas, offset+0, 0)
    
    footer(mycanvas, str(counters.page))
    mycanvas.showPage() # complete page
    counters.page += 1
    
    return canvas
    
    

if 0:
    VERBOSITY = 1
    logger.setLevel(logging.DEBUG)
    counters.reset()
    
    days = get_days(o)
    weeks = days.groupby(by='week_index')
    
    c = canvas.Canvas('test.pdf', pagesize = o.papersize)
    
    week = weeks.get_group(0)
    print(week)
    gen_week_landscape(o, week, c)
    
    c.save()
    os.system("start " + 'test.pdf')
    
# test across two sides of a single sheet of paper
if 1:
    VERBOSITY = 1
    logger.setLevel(logging.DEBUG)
    counters.reset()
    
    days = get_days(o)
    weeks = days.groupby(by='week_index')
    
    c = canvas.Canvas('test.pdf', pagesize = o.papersize)
    
    week = weeks.get_group(0)
    print(week)
    gen_week_landscape(o, week, c)
    
    week = weeks.get_group(1)
    print(week)
    gen_week_landscape(o, week, c)
    
    c.save()
    os.system("start " + 'test.pdf')

06/08/2018 06:31:14 PM - root - DEBUG - Initial date:2018-07-01, Sunday
06/08/2018 06:31:14 PM - root - DEBUG - 2018-07-01, Sunday
06/08/2018 06:31:14 PM - root - DEBUG - 2018-06-30, Saturday
06/08/2018 06:31:14 PM - root - DEBUG - 2018-06-29, Friday
06/08/2018 06:31:14 PM - root - DEBUG - 2018-06-28, Thursday
06/08/2018 06:31:14 PM - root - DEBUG - 2018-06-27, Wednesday
06/08/2018 06:31:14 PM - root - DEBUG - 2018-06-26, Tuesday
06/08/2018 06:31:14 PM - root - DEBUG - 2018-06-25, Monday
06/08/2018 06:31:14 PM - root - DEBUG - Initial date:, 2019-02-10, Sunday
06/08/2018 06:31:14 PM - root - DEBUG - 2019-02-10,Sunday
06/08/2018 06:31:14 PM - root - DEBUG - Days of the diary:
06/08/2018 06:31:14 PM - root - DEBUG - 0, 2018-06-25, Monday
06/08/2018 06:31:14 PM - root - DEBUG - 0, 2018-06-26, Tuesday
06/08/2018 06:31:14 PM - root - DEBUG - 0, 2018-06-27, Wednesday
06/08/2018 06:31:14 PM - root - DEBUG - 0, 2018-06-28, Thursday
06/08/2018 06:31:14 PM - root - DEBUG - 0, 2018-06-29, Friday


2018-06-25 2019-02-10


06/08/2018 06:31:15 PM - root - DEBUG - 4, 2018-07-28, Saturday
06/08/2018 06:31:15 PM - root - DEBUG - 4, 2018-07-29, Sunday
06/08/2018 06:31:15 PM - root - DEBUG - 5, 2018-07-30, Monday
06/08/2018 06:31:15 PM - root - DEBUG - 5, 2018-07-31, Tuesday
06/08/2018 06:31:15 PM - root - DEBUG - 5, 2018-08-01, Wednesday
06/08/2018 06:31:15 PM - root - DEBUG - 5, 2018-08-02, Thursday
06/08/2018 06:31:15 PM - root - DEBUG - 5, 2018-08-03, Friday
06/08/2018 06:31:15 PM - root - DEBUG - 5, 2018-08-04, Saturday
06/08/2018 06:31:15 PM - root - DEBUG - 5, 2018-08-05, Sunday
06/08/2018 06:31:15 PM - root - DEBUG - 6, 2018-08-06, Monday
06/08/2018 06:31:15 PM - root - DEBUG - 6, 2018-08-07, Tuesday
06/08/2018 06:31:15 PM - root - DEBUG - 6, 2018-08-08, Wednesday
06/08/2018 06:31:15 PM - root - DEBUG - 6, 2018-08-09, Thursday
06/08/2018 06:31:15 PM - root - DEBUG - 6, 2018-08-10, Friday
06/08/2018 06:31:15 PM - root - DEBUG - 6, 2018-08-11, Saturday
06/08/2018 06:31:15 PM - root - DEBUG - 6, 2018-08-1

06/08/2018 06:31:15 PM - root - DEBUG - 23, 2018-12-04, Tuesday
06/08/2018 06:31:15 PM - root - DEBUG - 23, 2018-12-05, Wednesday
06/08/2018 06:31:15 PM - root - DEBUG - 23, 2018-12-06, Thursday
06/08/2018 06:31:15 PM - root - DEBUG - 23, 2018-12-07, Friday
06/08/2018 06:31:15 PM - root - DEBUG - 23, 2018-12-08, Saturday
06/08/2018 06:31:15 PM - root - DEBUG - 23, 2018-12-09, Sunday
06/08/2018 06:31:15 PM - root - DEBUG - 24, 2018-12-10, Monday
06/08/2018 06:31:15 PM - root - DEBUG - 24, 2018-12-11, Tuesday
06/08/2018 06:31:16 PM - root - DEBUG - 24, 2018-12-12, Wednesday
06/08/2018 06:31:16 PM - root - DEBUG - 24, 2018-12-13, Thursday
06/08/2018 06:31:16 PM - root - DEBUG - 24, 2018-12-14, Friday
06/08/2018 06:31:16 PM - root - DEBUG - 24, 2018-12-15, Saturday
06/08/2018 06:31:16 PM - root - DEBUG - 24, 2018-12-16, Sunday
06/08/2018 06:31:16 PM - root - DEBUG - 25, 2018-12-17, Monday
06/08/2018 06:31:16 PM - root - DEBUG - 25, 2018-12-18, Tuesday
06/08/2018 06:31:16 PM - root - DEBUG 

         date  week_index    weekday  week_number  month_number month_name
0  2018-06-25           0     Monday           26             6       June
1  2018-06-26           0    Tuesday           26             6       June
2  2018-06-27           0  Wednesday           26             6       June
3  2018-06-28           0   Thursday           26             6       June
4  2018-06-29           0     Friday           26             6       June
5  2018-06-30           0   Saturday           26             6       June
6  2018-07-01           0     Sunday           26             7       July
          date  week_index    weekday  week_number  month_number month_name
7   2018-07-02           1     Monday           27             7       July
8   2018-07-03           1    Tuesday           27             7       July
9   2018-07-04           1  Wednesday           27             7       July
10  2018-07-05           1   Thursday           27             7       July
11  2018-07-06      

In [232]:
def gen_week_portrait(options, week, mycanvas):
    ''' Generate a single week
    
    Current design is 1 week on 2 pages.
    
    week ... list of days in the week, first is Monday, last is Sunday
    [datetime.date(2018, 4, 30), datetime.date(2018, 5, 1), datetime.date(2018, 5, 2), 
     datetime.date(2018, 5, 3), datetime.date(2018, 5, 4), datetime.date(2018, 5, 5), datetime.date(2018, 5, 6)]
    
    week=
             date       week_index  weekday      week_number month_number month_name
        0  2018-12-24           0     Monday           52            12   December
        1  2018-12-25           0    Tuesday           52            12   December
        2  2018-12-26           0  Wednesday           52            12   December
        3  2018-12-27           0   Thursday           52            12   December
        4  2018-12-28           0     Friday           52            12   December
        5  2018-12-29           0   Saturday           52            12   December
        6  2018-12-30           0     Sunday           52            12   December
    
    '''
    
    global counters
    
    logger.debug(week)
    
    week_section_options = {
        'top_margin' : 9*mm,
        'bottom_margin' : 9*mm,
        'top_box_height' : 3*cm,
        'weekly_tasks_width' : 5*cm,
        'weekly_tasks_row_height' : 6*mm,
        'weekly_focus_height' : 5*cm,
        'day_box_width' : 7*cm,
        'day_box_nlines' : 4, # number of lines inside each day box 
    }
    
    debug_print(0,height-10,f"Week section, week={week.date.iloc[0]}-{week.date.iloc[-1]}", mycanvas)
    
    mycanvas.drawString(0, height-20, f"Week number:{week.week_number.iloc[0]}")
    
    # boxes for days of the week
    day_box_height =( height-week_section_options['top_margin']-week_section_options['bottom_margin'] ) / 7
        
    # spacing of lines inside each day box
    day_box_line_spacing = day_box_height / (week_section_options['day_box_nlines']+1)
    logger.debug(day_box_line_spacing)
        
    for i, (ind, day) in enumerate( week.iterrows() ):
        day_name = day['weekday']
        logger.debug(f"{day.date}:{day_name}")
        mycanvas.grid([week_section_options['weekly_tasks_width'], 
                       width],
                      [height-week_section_options['top_margin']-i*day_box_height, 
                       height-week_section_options['top_margin']-(i+1)*day_box_height])
        # name of the day in the day box
        mycanvas.drawString(week_section_options['weekly_tasks_width'], 
                            height-week_section_options['top_margin']-i*day_box_height-(1)*day_box_line_spacing+1*mm,
                            f"{day_name[0:2]}:{day.date.day}.{day.date.month}.")

                                                 
        mycanvas.setStrokeColor('pink')
        for j in range( week_section_options['day_box_nlines'] ):
            mycanvas.line(week_section_options['weekly_tasks_width'],
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing, 
                          week_section_options['weekly_tasks_width'] + (width-week_section_options['weekly_tasks_width'])/2-0.5*cm,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing 
                         )
            mycanvas.line(week_section_options['weekly_tasks_width']+(width-week_section_options['weekly_tasks_width'])/2+0.5*cm,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing, 
                          width,
                          height-week_section_options['top_margin']-i*day_box_height-(j+1)*day_box_line_spacing 
                         )
        mycanvas.setStrokeColor('black')
                      
    
    # top box
    
    table = Table([[""]], colWidths=week_section_options['weekly_tasks_width'], 
                          rowHeights=week_section_options['top_box_height'])
    
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 1, colors.gray, None, (1,4)),
        ('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ]
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table width, height is {w, h}")
    table.drawOn(mycanvas, 0, height-week_section_options['top_box_height'])
    
    
    # left column with week tasks
    
    common_style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        #('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        ('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        ('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]

    # to-do: use a fix width font

    
    nr = int( (height-week_section_options['top_box_height']-week_section_options['weekly_focus_height'])//(6*mm) )

    table = Table(data=[ [""] for i in range(int(nr))], 
                  colWidths=[5*cm], rowHeights=nr*[week_section_options['weekly_tasks_row_height']]) 
    
    table.setStyle(TableStyle(common_style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table height is {h}")
    table.drawOn(mycanvas, 0, week_section_options['weekly_focus_height'])
    
    
    # weekly focus subsection
    table = Table([[""]], colWidths=week_section_options['weekly_tasks_width'], 
                          rowHeights=week_section_options['weekly_focus_height'])
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0),
          (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 1, colors.gray, None, (1,4)),
        ('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ]
    
    table.setStyle(TableStyle(style))
    w, h = table.wrapOn(mycanvas, 0, 0)
    logger.debug(f"table width, height is {w, h}")
    table.drawOn(mycanvas, 0, 0)
    
    footer(mycanvas, str(counters.page))
    mycanvas.showPage() # complete page
    counters.page += 1
    
    return canvas
    
    

if 1:
    VERBOSITY = 1
    logger.setLevel(logging.DEBUG)
    counters.reset()
    
    days = get_days(o)
    weeks = days.groupby(by='week_index')
    
    c = canvas.Canvas('test.pdf', pagesize=o.pagesize)
    
    week = weeks.get_group(0)
    print(week)
    gen_week(o, week, c)
    
    c.save()
    os.system("start " + 'test.pdf') 

06/08/2018 05:35:26 PM - root - DEBUG - Initial date:2018-07-01, Sunday
06/08/2018 05:35:26 PM - root - DEBUG - 2018-07-01, Sunday
06/08/2018 05:35:26 PM - root - DEBUG - 2018-06-30, Saturday
06/08/2018 05:35:26 PM - root - DEBUG - 2018-06-29, Friday
06/08/2018 05:35:26 PM - root - DEBUG - 2018-06-28, Thursday
06/08/2018 05:35:26 PM - root - DEBUG - 2018-06-27, Wednesday
06/08/2018 05:35:26 PM - root - DEBUG - 2018-06-26, Tuesday
06/08/2018 05:35:26 PM - root - DEBUG - 2018-06-25, Monday
06/08/2018 05:35:26 PM - root - DEBUG - Initial date:, 2019-02-10, Sunday
06/08/2018 05:35:26 PM - root - DEBUG - 2019-02-10,Sunday
06/08/2018 05:35:26 PM - root - DEBUG - Days of the diary:
06/08/2018 05:35:26 PM - root - DEBUG - 0, 2018-06-25, Monday


2018-06-25 2019-02-10


06/08/2018 05:35:26 PM - root - DEBUG - 0, 2018-06-26, Tuesday
06/08/2018 05:35:26 PM - root - DEBUG - 0, 2018-06-27, Wednesday
06/08/2018 05:35:26 PM - root - DEBUG - 0, 2018-06-28, Thursday
06/08/2018 05:35:26 PM - root - DEBUG - 0, 2018-06-29, Friday
06/08/2018 05:35:26 PM - root - DEBUG - 0, 2018-06-30, Saturday
06/08/2018 05:35:26 PM - root - DEBUG - 0, 2018-07-01, Sunday
06/08/2018 05:35:26 PM - root - DEBUG - 1, 2018-07-02, Monday
06/08/2018 05:35:26 PM - root - DEBUG - 1, 2018-07-03, Tuesday
06/08/2018 05:35:26 PM - root - DEBUG - 1, 2018-07-04, Wednesday
06/08/2018 05:35:26 PM - root - DEBUG - 1, 2018-07-05, Thursday
06/08/2018 05:35:26 PM - root - DEBUG - 1, 2018-07-06, Friday
06/08/2018 05:35:26 PM - root - DEBUG - 1, 2018-07-07, Saturday
06/08/2018 05:35:26 PM - root - DEBUG - 1, 2018-07-08, Sunday
06/08/2018 05:35:26 PM - root - DEBUG - 2, 2018-07-09, Monday
06/08/2018 05:35:26 PM - root - DEBUG - 2, 2018-07-10, Tuesday
06/08/2018 05:35:26 PM - root - DEBUG - 2, 2018-07-11

06/08/2018 05:35:27 PM - root - DEBUG - 18, 2018-11-02, Friday
06/08/2018 05:35:27 PM - root - DEBUG - 18, 2018-11-03, Saturday
06/08/2018 05:35:27 PM - root - DEBUG - 18, 2018-11-04, Sunday
06/08/2018 05:35:27 PM - root - DEBUG - 19, 2018-11-05, Monday
06/08/2018 05:35:27 PM - root - DEBUG - 19, 2018-11-06, Tuesday
06/08/2018 05:35:27 PM - root - DEBUG - 19, 2018-11-07, Wednesday
06/08/2018 05:35:27 PM - root - DEBUG - 19, 2018-11-08, Thursday
06/08/2018 05:35:27 PM - root - DEBUG - 19, 2018-11-09, Friday
06/08/2018 05:35:27 PM - root - DEBUG - 19, 2018-11-10, Saturday
06/08/2018 05:35:27 PM - root - DEBUG - 19, 2018-11-11, Sunday
06/08/2018 05:35:27 PM - root - DEBUG - 20, 2018-11-12, Monday
06/08/2018 05:35:27 PM - root - DEBUG - 20, 2018-11-13, Tuesday
06/08/2018 05:35:27 PM - root - DEBUG - 20, 2018-11-14, Wednesday
06/08/2018 05:35:27 PM - root - DEBUG - 20, 2018-11-15, Thursday
06/08/2018 05:35:27 PM - root - DEBUG - 20, 2018-11-16, Friday
06/08/2018 05:35:27 PM - root - DEBUG -

         date  week_index    weekday  week_number  month_number month_name
0  2018-06-25           0     Monday           26             6       June
1  2018-06-26           0    Tuesday           26             6       June
2  2018-06-27           0  Wednesday           26             6       June
3  2018-06-28           0   Thursday           26             6       June
4  2018-06-29           0     Friday           26             6       June
5  2018-06-30           0   Saturday           26             6       June
6  2018-07-01           0     Sunday           26             7       July


In [294]:
def gen_weeks(options, canvas):
    
    
    logger.info("Generating weeks")
    
    days = get_days(options)
    weeks = days.groupby(by='week_index')

    logger.info(f"Starting weeks section from {days.iloc[0]['date']} to {days.iloc[-1]['date']}")
    
    debug_print(10,10,
                f"Starting weeks section from {days.iloc[0]['date']} to {days.iloc[-1]['date']}",
                canvas)

    for w, week in weeks:
    
        logger.debug(f"week {w}")
        debug_print(10,20, f"week index={w},{week.week_index.iloc[0]}", canvas)
        gen_week_landscape(options, week, canvas)
        counters.week += 1

if 1:
    
    VERBOSITY = 1
    
    logger.setLevel(logging.DEBUG)
    
    counters.reset() # reset page counters
    
    c = canvas.Canvas('test.pdf', pagesize= o.papersize)
    
    test_options = copy.deepcopy(o)
    test_options.daterange['start_date']=datetime.date(2018, 12, 24)
    test_options.daterange['end_date']=datetime.date(2019, 6, 10)
    
    print(test_options)
    
    gen_weeks(test_options, c)

    c.save()
    os.system("start " + 'test.pdf') 
    


06/08/2018 06:31:51 PM - root - INFO - Generating weeks
06/08/2018 06:31:51 PM - root - DEBUG - Initial date:2018-12-24, Monday
06/08/2018 06:31:51 PM - root - DEBUG - 2018-12-24, Monday
06/08/2018 06:31:51 PM - root - DEBUG - Initial date:, 2019-06-10, Monday
06/08/2018 06:31:51 PM - root - DEBUG - 2019-06-10,Monday
06/08/2018 06:31:51 PM - root - DEBUG - 2019-06-11,Tuesday
06/08/2018 06:31:51 PM - root - DEBUG - 2019-06-12,Wednesday
06/08/2018 06:31:51 PM - root - DEBUG - 2019-06-13,Thursday
06/08/2018 06:31:51 PM - root - DEBUG - 2019-06-14,Friday
06/08/2018 06:31:51 PM - root - DEBUG - 2019-06-15,Saturday
06/08/2018 06:31:51 PM - root - DEBUG - 2019-06-16,Sunday
06/08/2018 06:31:51 PM - root - DEBUG - Days of the diary:
06/08/2018 06:31:51 PM - root - DEBUG - 0, 2018-12-24, Monday
06/08/2018 06:31:51 PM - root - DEBUG - 0, 2018-12-25, Tuesday
06/08/2018 06:31:51 PM - root - DEBUG - 0, 2018-12-26, Wednesday
06/08/2018 06:31:51 PM - root - DEBUG - 0, 2018-12-27, Thursday
06/08/2018 0

debug_verbosity:1
layout:1
format:<built-in function format>
daterange:{'start_date': datetime.date(2018, 12, 24), 'end_date': datetime.date(2019, 6, 10)}
monthrange:{'start': datetime.date(2018, 7, 1), 'stop': datetime.date(2019, 2, 10)}
sections:['info_sheet', 'months_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'weeks_v2_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout']
pagesize:(445.03937007874015, 595.275590551181)
papersize:[841.8897637795275, 595.275590551181]
margins:(14.173228346456693, 14.173228346456693, 14.173228346456693, 14.173228346456693)

2018-12-24 2019-06-16


06/08/2018 06:31:51 PM - root - DEBUG - 4, 2019-01-24, Thursday
06/08/2018 06:31:51 PM - root - DEBUG - 4, 2019-01-25, Friday
06/08/2018 06:31:51 PM - root - DEBUG - 4, 2019-01-26, Saturday
06/08/2018 06:31:51 PM - root - DEBUG - 4, 2019-01-27, Sunday
06/08/2018 06:31:51 PM - root - DEBUG - 5, 2019-01-28, Monday
06/08/2018 06:31:51 PM - root - DEBUG - 5, 2019-01-29, Tuesday
06/08/2018 06:31:51 PM - root - DEBUG - 5, 2019-01-30, Wednesday
06/08/2018 06:31:51 PM - root - DEBUG - 5, 2019-01-31, Thursday
06/08/2018 06:31:51 PM - root - DEBUG - 5, 2019-02-01, Friday
06/08/2018 06:31:51 PM - root - DEBUG - 5, 2019-02-02, Saturday
06/08/2018 06:31:51 PM - root - DEBUG - 5, 2019-02-03, Sunday
06/08/2018 06:31:51 PM - root - DEBUG - 6, 2019-02-04, Monday
06/08/2018 06:31:51 PM - root - DEBUG - 6, 2019-02-05, Tuesday
06/08/2018 06:31:51 PM - root - DEBUG - 6, 2019-02-06, Wednesday
06/08/2018 06:31:51 PM - root - DEBUG - 6, 2019-02-07, Thursday
06/08/2018 06:31:51 PM - root - DEBUG - 6, 2019-02-0

06/08/2018 06:31:52 PM - root - DEBUG - 22, 2019-06-02, Sunday
06/08/2018 06:31:52 PM - root - DEBUG - 23, 2019-06-03, Monday
06/08/2018 06:31:52 PM - root - DEBUG - 23, 2019-06-04, Tuesday
06/08/2018 06:31:52 PM - root - DEBUG - 23, 2019-06-05, Wednesday
06/08/2018 06:31:52 PM - root - DEBUG - 23, 2019-06-06, Thursday
06/08/2018 06:31:52 PM - root - DEBUG - 23, 2019-06-07, Friday
06/08/2018 06:31:52 PM - root - DEBUG - 23, 2019-06-08, Saturday
06/08/2018 06:31:52 PM - root - DEBUG - 23, 2019-06-09, Sunday
06/08/2018 06:31:52 PM - root - DEBUG - 24, 2019-06-10, Monday
06/08/2018 06:31:52 PM - root - DEBUG - 24, 2019-06-11, Tuesday
06/08/2018 06:31:52 PM - root - DEBUG - 24, 2019-06-12, Wednesday
06/08/2018 06:31:52 PM - root - DEBUG - 24, 2019-06-13, Thursday
06/08/2018 06:31:52 PM - root - DEBUG - 24, 2019-06-14, Friday
06/08/2018 06:31:52 PM - root - DEBUG - 24, 2019-06-15, Saturday
06/08/2018 06:31:52 PM - root - DEBUG - 24, 2019-06-16, Sunday
06/08/2018 06:31:52 PM - root - INFO - 

06/08/2018 06:31:52 PM - root - DEBUG - week 5
06/08/2018 06:31:52 PM - root - DEBUG -           date  week_index    weekday  week_number  month_number month_name
35  2019-01-28           5     Monday            5             1    January
36  2019-01-29           5    Tuesday            5             1    January
37  2019-01-30           5  Wednesday            5             1    January
38  2019-01-31           5   Thursday            5             1    January
39  2019-02-01           5     Friday            5             2   February
40  2019-02-02           5   Saturday            5             2   February
41  2019-02-03           5     Sunday            5             2   February
06/08/2018 06:31:52 PM - root - DEBUG - 15.550056242969626
06/08/2018 06:31:52 PM - root - DEBUG - 2019-01-28:Monday
06/08/2018 06:31:52 PM - root - DEBUG - 2019-01-29:Tuesday
06/08/2018 06:31:52 PM - root - DEBUG - 2019-01-30:Wednesday
06/08/2018 06:31:52 PM - root - DEBUG - 2019-01-31:Thursday
06/08/20

06/08/2018 06:31:52 PM - root - DEBUG - 2019-03-08:Friday
06/08/2018 06:31:52 PM - root - DEBUG - 2019-03-09:Saturday
06/08/2018 06:31:52 PM - root - DEBUG - 2019-03-10:Sunday
06/08/2018 06:31:52 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
06/08/2018 06:31:52 PM - root - DEBUG - table height is 357.1653543307087
06/08/2018 06:31:52 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
06/08/2018 06:31:52 PM - root - DEBUG - week 11
06/08/2018 06:31:52 PM - root - DEBUG -           date  week_index    weekday  week_number  month_number month_name
77  2019-03-11          11     Monday           11             3      March
78  2019-03-12          11    Tuesday           11             3      March
79  2019-03-13          11  Wednesday           11             3      March
80  2019-03-14          11   Thursday           11             3      March
81  2019-03-15          11     Friday           11             3      March

06/08/2018 06:31:53 PM - root - DEBUG - 15.550056242969626
06/08/2018 06:31:53 PM - root - DEBUG - 2019-04-15:Monday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-04-16:Tuesday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-04-17:Wednesday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-04-18:Thursday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-04-19:Friday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-04-20:Saturday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-04-21:Sunday
06/08/2018 06:31:53 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
06/08/2018 06:31:53 PM - root - DEBUG - table height is 357.1653543307087
06/08/2018 06:31:53 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
06/08/2018 06:31:53 PM - root - DEBUG - week 17
06/08/2018 06:31:53 PM - root - DEBUG -            date  week_index    weekday  week_number  month_number month_name
119  2019-04-22          17     Monday           17             4      April
120 

06/08/2018 06:31:53 PM - root - DEBUG - 15.550056242969626
06/08/2018 06:31:53 PM - root - DEBUG - 2019-05-27:Monday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-05-28:Tuesday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-05-29:Wednesday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-05-30:Thursday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-05-31:Friday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-06-01:Saturday
06/08/2018 06:31:53 PM - root - DEBUG - 2019-06-02:Sunday
06/08/2018 06:31:53 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
06/08/2018 06:31:53 PM - root - DEBUG - table height is 357.1653543307087
06/08/2018 06:31:53 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
06/08/2018 06:31:53 PM - root - DEBUG - week 23
06/08/2018 06:31:53 PM - root - DEBUG -            date  week_index    weekday  week_number  month_number month_name
161  2019-06-03          23     Monday           23             6       June
162 

In [313]:
def gen_weeks_withmonthly_notes(options, canvas):

    logger.info("Generating weeks with monthly notes")
    
    days = get_days(options)
    weeks = days.groupby(by='week_index')

    logger.info(f"Starting weeks section from {days.iloc[0]['date']} to {days.iloc[-1]['date']}")
    
    debug_print(10,10,
                f"Starting weeks section from {days.iloc[0]['date']} to {days.iloc[-1]['date']}",
                canvas)

    for w, week in weeks:
    
        logger.debug(f"week {w}")
        logger.debug(30*"=")
        logger.debug(f"week")
        debug_print(10,20, f"week index={w},{week.week_index.iloc[0]}", canvas)
        gen_week_landscape(options, week, canvas)
        counters.week += 1
        
        # Add notes pages at the end of each month, more precisely, add notes pages after the last week of each month
        # This is done by detecting that the last week had the month's last day in it
        
        last_week_first_day_month =  (week.date.iloc[0]).month # month to which the first day of the last week belongs
        logger.debug(f"First day of this week belongs to month {last_week_first_day_month}")
        
        last_day_of_month = datetime.date((week.date.iloc[0]).year, last_week_first_day_month, 
                                          calendar.monthrange((week.date.iloc[0]).year, last_week_first_day_month)[1])
        
        logger.debug([week.date==last_day_of_month])
        
        if (week.date==last_day_of_month).any():
            # add notes pages
            pass
            #gen_notes_landscape(options, canvas)
            #gen_notes_landscape(options, canvas)
            #gen_notes_landscape(options, canvas)
        
if 1:
    
    VERBOSITY = 1
    
    logger.setLevel(logging.DEBUG)
    
    counters.reset() # reset page counters
    
    c = canvas.Canvas('test.pdf', pagesize = o.papersize)
    
    test_options = copy.deepcopy(o)
    test_options.daterange['start_date']=datetime.date(2018, 12, 24)
    test_options.daterange['end_date']=datetime.date(2019, 6, 10)
    
    print(test_options)
    
    gen_weeks_withmonthly_notes(test_options, c)

    c.save()
    os.system("start " + 'test.pdf')     

06/08/2018 07:26:22 PM - root - INFO - Generating weeks with monthly notes
06/08/2018 07:26:22 PM - root - DEBUG - Initial date:2018-12-24, Monday
06/08/2018 07:26:22 PM - root - DEBUG - 2018-12-24, Monday
06/08/2018 07:26:22 PM - root - DEBUG - Initial date:, 2019-06-10, Monday
06/08/2018 07:26:22 PM - root - DEBUG - 2019-06-10,Monday
06/08/2018 07:26:22 PM - root - DEBUG - 2019-06-11,Tuesday
06/08/2018 07:26:22 PM - root - DEBUG - 2019-06-12,Wednesday
06/08/2018 07:26:22 PM - root - DEBUG - 2019-06-13,Thursday
06/08/2018 07:26:22 PM - root - DEBUG - 2019-06-14,Friday
06/08/2018 07:26:22 PM - root - DEBUG - 2019-06-15,Saturday
06/08/2018 07:26:22 PM - root - DEBUG - 2019-06-16,Sunday
06/08/2018 07:26:22 PM - root - DEBUG - Days of the diary:
06/08/2018 07:26:22 PM - root - DEBUG - 0, 2018-12-24, Monday
06/08/2018 07:26:22 PM - root - DEBUG - 0, 2018-12-25, Tuesday
06/08/2018 07:26:22 PM - root - DEBUG - 0, 2018-12-26, Wednesday
06/08/2018 07:26:22 PM - root - DEBUG - 0, 2018-12-27, Th

debug_verbosity:1
layout:1
format:<built-in function format>
daterange:{'start_date': datetime.date(2018, 12, 24), 'end_date': datetime.date(2019, 6, 10)}
monthrange:{'start': datetime.date(2018, 5, 1), 'stop': datetime.date(2019, 2, 10)}
sections:['info_sheet', 'months_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'weeks_v2_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout']
pagesize:(445.03937007874015, 595.275590551181)
papersize:[841.8897637795275, 595.275590551181]
margins:(14.173228346456693, 14.173228346456693, 14.173228346456693, 14.173228346456693)

2018-12-24 2019-06-16


06/08/2018 07:26:22 PM - root - DEBUG - 4, 2019-01-23, Wednesday
06/08/2018 07:26:22 PM - root - DEBUG - 4, 2019-01-24, Thursday
06/08/2018 07:26:22 PM - root - DEBUG - 4, 2019-01-25, Friday
06/08/2018 07:26:22 PM - root - DEBUG - 4, 2019-01-26, Saturday
06/08/2018 07:26:22 PM - root - DEBUG - 4, 2019-01-27, Sunday
06/08/2018 07:26:22 PM - root - DEBUG - 5, 2019-01-28, Monday
06/08/2018 07:26:22 PM - root - DEBUG - 5, 2019-01-29, Tuesday
06/08/2018 07:26:22 PM - root - DEBUG - 5, 2019-01-30, Wednesday
06/08/2018 07:26:22 PM - root - DEBUG - 5, 2019-01-31, Thursday
06/08/2018 07:26:22 PM - root - DEBUG - 5, 2019-02-01, Friday
06/08/2018 07:26:22 PM - root - DEBUG - 5, 2019-02-02, Saturday
06/08/2018 07:26:22 PM - root - DEBUG - 5, 2019-02-03, Sunday
06/08/2018 07:26:22 PM - root - DEBUG - 6, 2019-02-04, Monday
06/08/2018 07:26:22 PM - root - DEBUG - 6, 2019-02-05, Tuesday
06/08/2018 07:26:22 PM - root - DEBUG - 6, 2019-02-06, Wednesday
06/08/2018 07:26:22 PM - root - DEBUG - 6, 2019-02-

06/08/2018 07:26:23 PM - root - DEBUG - 22, 2019-06-01, Saturday
06/08/2018 07:26:23 PM - root - DEBUG - 22, 2019-06-02, Sunday
06/08/2018 07:26:23 PM - root - DEBUG - 23, 2019-06-03, Monday
06/08/2018 07:26:23 PM - root - DEBUG - 23, 2019-06-04, Tuesday
06/08/2018 07:26:23 PM - root - DEBUG - 23, 2019-06-05, Wednesday
06/08/2018 07:26:23 PM - root - DEBUG - 23, 2019-06-06, Thursday
06/08/2018 07:26:23 PM - root - DEBUG - 23, 2019-06-07, Friday
06/08/2018 07:26:23 PM - root - DEBUG - 23, 2019-06-08, Saturday
06/08/2018 07:26:23 PM - root - DEBUG - 23, 2019-06-09, Sunday
06/08/2018 07:26:23 PM - root - DEBUG - 24, 2019-06-10, Monday
06/08/2018 07:26:23 PM - root - DEBUG - 24, 2019-06-11, Tuesday
06/08/2018 07:26:23 PM - root - DEBUG - 24, 2019-06-12, Wednesday
06/08/2018 07:26:23 PM - root - DEBUG - 24, 2019-06-13, Thursday
06/08/2018 07:26:23 PM - root - DEBUG - 24, 2019-06-14, Friday
06/08/2018 07:26:23 PM - root - DEBUG - 24, 2019-06-15, Saturday
06/08/2018 07:26:23 PM - root - DEBUG

06/08/2018 07:26:23 PM - root - DEBUG - week 4
06/08/2018 07:26:23 PM - root - DEBUG - ==============================
06/08/2018 07:26:23 PM - root - DEBUG - week
06/08/2018 07:26:23 PM - root - DEBUG -           date  week_index    weekday  week_number  month_number month_name
28  2019-01-21           4     Monday            4             1    January
29  2019-01-22           4    Tuesday            4             1    January
30  2019-01-23           4  Wednesday            4             1    January
31  2019-01-24           4   Thursday            4             1    January
32  2019-01-25           4     Friday            4             1    January
33  2019-01-26           4   Saturday            4             1    January
34  2019-01-27           4     Sunday            4             1    January
06/08/2018 07:26:23 PM - root - DEBUG - 15.550056242969626
06/08/2018 07:26:23 PM - root - DEBUG - 2019-01-21:Monday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-01-22:Tuesday
06/08/2018 07

06/08/2018 07:26:23 PM - root - DEBUG - 2019-02-21:Thursday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-02-22:Friday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-02-23:Saturday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-02-24:Sunday
06/08/2018 07:26:23 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
06/08/2018 07:26:23 PM - root - DEBUG - table height is 357.1653543307087
06/08/2018 07:26:23 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
06/08/2018 07:26:23 PM - root - DEBUG - First day of this week belongs to month 2
06/08/2018 07:26:23 PM - root - DEBUG - [56    False
57    False
58    False
59    False
60    False
61    False
62    False
Name: date, dtype: bool]
06/08/2018 07:26:23 PM - root - DEBUG - week 9
06/08/2018 07:26:23 PM - root - DEBUG - ==============================
06/08/2018 07:26:23 PM - root - DEBUG - week
06/08/2018 07:26:23 PM - root - DEBUG -           date  week_index    weekday  week_numb

06/08/2018 07:26:23 PM - root - DEBUG - 15.550056242969626
06/08/2018 07:26:23 PM - root - DEBUG - 2019-03-25:Monday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-03-26:Tuesday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-03-27:Wednesday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-03-28:Thursday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-03-29:Friday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-03-30:Saturday
06/08/2018 07:26:23 PM - root - DEBUG - 2019-03-31:Sunday
06/08/2018 07:26:23 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
06/08/2018 07:26:23 PM - root - DEBUG - table height is 357.1653543307087
06/08/2018 07:26:23 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
06/08/2018 07:26:23 PM - root - DEBUG - First day of this week belongs to month 3
06/08/2018 07:26:23 PM - root - DEBUG - [91    False
92    False
93    False
94    False
95    False
96    False
97     True
Name: date, dtype: bool]
06/08/2018 07:

06/08/2018 07:26:24 PM - root - DEBUG - week 18
06/08/2018 07:26:24 PM - root - DEBUG - ==============================
06/08/2018 07:26:24 PM - root - DEBUG - week
06/08/2018 07:26:24 PM - root - DEBUG -            date  week_index    weekday  week_number  month_number month_name
126  2019-04-29          18     Monday           18             4      April
127  2019-04-30          18    Tuesday           18             4      April
128  2019-05-01          18  Wednesday           18             5        May
129  2019-05-02          18   Thursday           18             5        May
130  2019-05-03          18     Friday           18             5        May
131  2019-05-04          18   Saturday           18             5        May
132  2019-05-05          18     Sunday           18             5        May
06/08/2018 07:26:24 PM - root - DEBUG - 15.550056242969626
06/08/2018 07:26:24 PM - root - DEBUG - 2019-04-29:Monday
06/08/2018 07:26:24 PM - root - DEBUG - 2019-04-30:Tuesday
06/0

06/08/2018 07:26:24 PM - root - DEBUG - 2019-05-29:Wednesday
06/08/2018 07:26:24 PM - root - DEBUG - 2019-05-30:Thursday
06/08/2018 07:26:24 PM - root - DEBUG - 2019-05-31:Friday
06/08/2018 07:26:24 PM - root - DEBUG - 2019-06-01:Saturday
06/08/2018 07:26:24 PM - root - DEBUG - 2019-06-02:Sunday
06/08/2018 07:26:24 PM - root - DEBUG - table width, height is (141.73228346456693, 85.03937007874015)
06/08/2018 07:26:24 PM - root - DEBUG - table height is 357.1653543307087
06/08/2018 07:26:24 PM - root - DEBUG - table width, height is (141.73228346456693, 141.73228346456693)
06/08/2018 07:26:24 PM - root - DEBUG - First day of this week belongs to month 5
06/08/2018 07:26:24 PM - root - DEBUG - [154    False
155    False
156    False
157    False
158     True
159    False
160    False
Name: date, dtype: bool]
06/08/2018 07:26:24 PM - root - DEBUG - week 23
06/08/2018 07:26:24 PM - root - DEBUG - ==============================
06/08/2018 07:26:24 PM - root - DEBUG - week
06/08/2018 07:26:24

In [298]:
def gen_notes_landscape(options, canvas):
    ''' Generate notes pages'''
    
    logger.info("Generating notes section")
    debug_print(0,height-30,"debug:Notes section",canvas)
    
    offset = get_offset(counters.page, options)
    
    draw_paperframe(canvas, options, offset)
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray, None, (1,4)),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        #('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        #('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        #('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        #('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]
    
    # make as many rows as columns as to fill the page
    row_size = 9*mm
    col_size = 9*mm
    
    rows = int( (pheight-30) // row_size )
    logger.debug(f"Rows:{rows}")
    
    horizontal_margin = 30
    
    cols = int( (pwidth-2*horizontal_margin) // col_size )
    logger.debug(f"Cols:{cols}")
    
    table_data = [ ['' for i in range(cols)] for i in range(rows) ]
    table = Table(table_data, colWidths=cols*[col_size], rowHeights=rows*[row_size])
    logger.debug(f"{rows}, {cols}")
    table.setStyle(TableStyle(style))
    
    w, h = table.wrapOn(canvas, 0, 0)
    table.drawOn(canvas, offset+horizontal_margin, 0)
    
    footer(canvas, str(counters.page))
    canvas.showPage() 
    counters.page += 1

if 1:
    logger.setLevel(logging.DEBUG)
    
    counters.reset()
    
    c = canvas.Canvas('test.pdf', pagesize = o.papersize)

    gen_notes_landscape(o, c)
    gen_notes_landscape(o, c)
    gen_notes_landscape(o, c)
    gen_notes_landscape(o, c)
    
    c.save()
    os.system("start " + 'test.pdf')    

06/08/2018 06:33:47 PM - root - INFO - Generating notes section
06/08/2018 06:33:47 PM - root - DEBUG - Rows:22
06/08/2018 06:33:47 PM - root - DEBUG - Cols:15
06/08/2018 06:33:47 PM - root - DEBUG - 22, 15
06/08/2018 06:33:47 PM - root - INFO - Generating notes section
06/08/2018 06:33:47 PM - root - DEBUG - Rows:22
06/08/2018 06:33:47 PM - root - DEBUG - Cols:15
06/08/2018 06:33:47 PM - root - DEBUG - 22, 15
06/08/2018 06:33:47 PM - root - INFO - Generating notes section
06/08/2018 06:33:47 PM - root - DEBUG - Rows:22
06/08/2018 06:33:47 PM - root - DEBUG - Cols:15
06/08/2018 06:33:47 PM - root - DEBUG - 22, 15
06/08/2018 06:33:47 PM - root - INFO - Generating notes section
06/08/2018 06:33:47 PM - root - DEBUG - Rows:22
06/08/2018 06:33:47 PM - root - DEBUG - Cols:15
06/08/2018 06:33:47 PM - root - DEBUG - 22, 15


In [299]:
def gen_notes(options, canvas):
    ''' Generate notes pages'''
    
    logger.info("Generating notes section")
    canvas.drawString(0,height-30,"debug:Notes section")
    
    style = [
        ('FONT', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('INNERGRID', (0, 0), (-1, -1), 0.01, colors.gray, None, (1,4)),
        #('BOX', (0, 0), (-1, -1), 0.01, colors.gray),
        #('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        #('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        #('LINEABOVE', (0,1), (1,-1), 0.1, colors.gray, None, (1,4)),
        #('LINEBEFORE', (0,0), (0,-1), -1, colors.white), # verical line
        #('LINEAFTER', (0,0), (0,-1), -1, colors.pink) # vertical line
    ]
    
    # make as many rows as columns as to fill the page
    row_size = 9*mm
    col_size = 9*mm
    
    rows = int( (height-30) // row_size )
    logger.debug(f"Rows:{rows}")
    
    horizontal_margin = 30
    
    cols = int( (width-2*horizontal_margin) // col_size )
    logger.debug(f"Cols:{cols}")
    
    table_data = [ ['' for i in range(cols)] for i in range(rows) ]
    table = Table(table_data, colWidths=cols*[col_size], rowHeights=rows*[row_size])
    logger.debug(f"{rows}, {cols}")
    table.setStyle(TableStyle(style))
    
    w, h = table.wrapOn(canvas, 0, 0)
    table.drawOn(canvas, horizontal_margin, 0)
    
    footer(canvas, str(counters.page))
    canvas.showPage() 
    counters.page += 1

if 1:
    logger.setLevel(logging.DEBUG)
    
    counters.reset()
    
    c = canvas.Canvas('test.pdf', pagesize = o.papersize)

    gen_notes(o, c)
    
    c.save()
    os.system("start " + 'test.pdf')    

06/08/2018 06:34:01 PM - root - INFO - Generating notes section
06/08/2018 06:34:01 PM - root - DEBUG - Rows:22
06/08/2018 06:34:01 PM - root - DEBUG - Cols:30
06/08/2018 06:34:01 PM - root - DEBUG - 22, 30


In [301]:
def gen_info_sheet(options, canvas):
    
    x = 10
    row_height = 20
    
    canvas.drawString(x,height-row_height,"INFO SHEET")
    
    i = 0
    for a in  dir(options):
        if a.startswith('__'):
            continue
        print(options.__getattribute__(a))
        canvas.drawString(x,height-(i+2)*row_height, \
            f"{a}::{str(options.__getattribute__(a))}")
        i += 1

    
    pass # no page number
    canvas.showPage()


if 1:
    c = canvas.Canvas('test.pdf', pageCompression = o.papersize)
    gen_info_sheet(o, c)
    c.save()
    os.system("start " + 'test.pdf')

{'start_date': datetime.date(2018, 7, 1), 'end_date': datetime.date(2019, 2, 10)}
1
<built-in function format>
1
(14.173228346456693, 14.173228346456693, 14.173228346456693, 14.173228346456693)
{'start': datetime.date(2018, 7, 1), 'stop': datetime.date(2019, 2, 10)}
(445.03937007874015, 595.275590551181)
[841.8897637795275, 595.275590551181]
['info_sheet', 'months_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'weeks_v2_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout']


# Generate calendar

In [314]:

section_generators = {
    'info_sheet' : gen_info_sheet, 
    'months_v1'  : gen_months_section,
    'weeks_v1'   : gen_weeks,
    'weeks_v2'   : gen_weeks_withmonthly_notes,
    'notes_v1'   : gen_notes,
    'months_v1_landscape_cutout' : gen_months_section,
    'weeks_v2_landscape_cutout' : gen_weeks_withmonthly_notes,
    'notes_v1_landscape_cutout' : gen_notes_landscape,
     
}

VERBOSITY = 0

logger.setLevel(logging.INFO)

counters.reset()

c = canvas.Canvas('diary.pdf', pagesize = o.papersize)

for section in o.sections:
    logger.info(section)
    section_generators[section](o, c) # calls a generator as per dictionary
    
c.save()

os.system("start " + 'diary.pdf')

06/08/2018 07:26:33 PM - root - INFO - info_sheet
06/08/2018 07:26:33 PM - root - INFO - months_v1_landscape_cutout
06/08/2018 07:26:33 PM - root - INFO - Number of months = 10
06/08/2018 07:26:33 PM - root - INFO - Even number of months.
06/08/2018 07:26:33 PM - root - INFO - Months section generated for the following list of months:[(2018, 5), (2018, 6), (2018, 7), (2018, 8), (2018, 9), (2018, 10), (2018, 11), (2018, 12), (2019, 1), (2019, 2)]
06/08/2018 07:26:33 PM - root - INFO - Generating months (2018, 5) and (2018, 6)
06/08/2018 07:26:33 PM - root - INFO - Page number:0
06/08/2018 07:26:33 PM - root - INFO - Generating months (2018, 7) and (2018, 8)
06/08/2018 07:26:33 PM - root - INFO - Page number:1
06/08/2018 07:26:33 PM - root - INFO - Generating months (2018, 9) and (2018, 10)
06/08/2018 07:26:33 PM - root - INFO - Page number:2
06/08/2018 07:26:33 PM - root - INFO - Generating months (2018, 11) and (2018, 12)
06/08/2018 07:26:33 PM - root - INFO - Page number:3
06/08/2018 

{'start_date': datetime.date(2018, 5, 1), 'end_date': datetime.date(2019, 2, 10)}
1
<built-in function format>
1
(14.173228346456693, 14.173228346456693, 14.173228346456693, 14.173228346456693)
{'start': datetime.date(2018, 5, 1), 'stop': datetime.date(2019, 2, 10)}
(445.03937007874015, 595.275590551181)
[841.8897637795275, 595.275590551181]
['info_sheet', 'months_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'weeks_v2_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout', 'notes_v1_landscape_cutout']
2018-04-30 2019-02-10


06/08/2018 07:26:34 PM - root - INFO - Starting weeks section from 2018-04-30 to 2019-02-10
06/08/2018 07:26:34 PM - root - INFO - notes_v1_landscape_cutout
06/08/2018 07:26:34 PM - root - INFO - Generating notes section
06/08/2018 07:26:34 PM - root - INFO - notes_v1_landscape_cutout
06/08/2018 07:26:34 PM - root - INFO - Generating notes section
06/08/2018 07:26:34 PM - root - INFO - notes_v1_landscape_cutout
06/08/2018 07:26:34 PM - root - INFO - Generating notes section
06/08/2018 07:26:34 PM - root - INFO - notes_v1_landscape_cutout
06/08/2018 07:26:34 PM - root - INFO - Generating notes section


0